<h2 style="text-align:center;color:#0F4C81;">
N-Gram Language Models
</h2>

Predicting is difficult—especially about the future, as the old quip goes. But how about predicting something that seems much easier, like the next word someone is going to say? What word, for example, is likely to follow:

<pre>
The water of Walden Pond is so beautifully ...
</pre>

You might conclude that a likely word is _blue_, or _green_, or _clear_, but probably not _refrigerator_ nor _this_. 
A **language model** is a machine learning model LM that predicts upcoming words. More formally, a language model assigns a probability to each possible next word, or equivalently gives a probability distribution over possible next works. Language models can also assign a probability to an entire sentence. Thus an LM could tell us that the following sequence has a much higher probability of appearing in a text:

<pre>
all of a sudden I notice three guys standing on the sidewalk
</pre>

than does this same set of words in a different order:

<pre>
on guys all I of notice sidewalk three a sudden standing the
</pre>

Why would we want to predict upcoming words, or know the probability of a sentence? One reason is for generation: choosing contextually better words. For example we can correct grammar or spelling errors like _Their are two midterms_, in which _There_ was mistyped as _Their_, or _Everything has improve_, in which improve should have been _improved_. The phrase _There are_ is more probable
than _Their are_, and _has improved_ than _has improve_, so a language model can help users select the more grammatical variant. Or for a speech system to recognize that you said _I will be back soonish_ and not _I will be bassoon dish_, it helps to know that _back soonish_ is a more probable sequence. Language models can also help in **augmentative and alternative communication** (Trnka et al. 2007, AAC Kane et al. 2017). People can use **AAC** systems if they are physically unable to speak or sign but can instead use eye gaze or other movements to select words from a menu. Word prediction can be used to suggest likely words for the menu.

Word prediction is also central to NLP for another reason: **large language models** are built just by training them to predict words!! Large language models learn an enormous amount about language solely from being trained to predict upcoming words from neighboring words.

### N-grams

An **n-gram** is a sequence of n words: a 2-gram (which we’ll call **bigram**) is a two-word sequence of words like _The water_, or _water of_, and a 3-gram (a **trigram**) is a three-word sequence of words like _The water of_, or _water of Walden_. But we also use the word ‘ngram’ to mean a probabilistic model that can estimate the probability of a word given the $n-1$ previous words, and thereby also to assign probabilities to entire sequences.

Let’s begin with the task of computing $P(w|h)$, the probability of a word $w$ given some history $h$. Suppose the history $h$ is _“The water of Walden Pond is so beautifully”_ and we want to know the probability that the next word is _blue_:

$$
P(\text{blue}|\text{The water of Walden Pond is so beautifully})
$$

One way to estimate this probability is directly from relative frequency counts: $1.$ take a very large corpus, $2.$ count the number of times we see _The water of Walden Pond is so beautifully_, and $3.$ count the number of times this is followed by _blue_. This would be answering the question “Out of the times we saw the history $h$, how many times was it followed by the word $w$”, as follows:

$$
P(\text{blue}|\text{The water of Walden Pond is so beautifully}) = \\
= \frac{C(\text{The water of Walden Pond is so beautifully blue})}{C(\text{The water of Walden Pond is so beautifully})}
$$

If we had a large enough corpus, we could compute these two counts and estimate the probability using the equation abovr. But even the entire web isn’t big enough to give us good estimates for counts of entire sentences. This is because language is creative; new sentences are invented all the time, and we can’t expect to get accurate counts for such large objects as entire sentences. For this reason, we’ll need more clever ways to estimate the probability of a word $w$ given a history $h$, or the probability of an entire word sequence $W$.

Now, how can we compute probabilities of entire sequences like $P(w_1, w_2, \dots, w_n)$? One thing we can do is decompose this probability using the **chain rule of probability**:

$$
P(X_1 \cdots X_n) = P(X_1)P(X_2|X_1)P(X_3|X_{1:2}) \dots P(X_n|X{1:n-1}) = \prod_{k=1}^n P(X_k|X_{1:k-1})
$$

Applying the chain rule to words, we get

$$
P(w_{1:n}) = P(w_1)P(w_2|w_1)P(w_3|w_{1:2}) \cdots P(w_n|w_{1:n-1}) = \prod_{k=1}^n P(w_k|w_{1:k-1})
$$

But using the chain rule doesn’t really seem to help us! We don’t know any way to compute the exact probability of a word given a long sequence of preceding words, $P(w_n|w_{1:n-1})$. As we said above, we can’t just estimate by counting the number of times every word occurs following every long string in some corpus, because language is creative and any particular context might have never occurred before!

### The Markov assumption

The intuition of the n-gram model is that instead of computing the probability of a word given its entire history, we can **approximate** the history by just the last few words.

The **bigram** model, for example, approximates the probability of a word given all the previous words $P(w_n|w_{1:n-1})$ by using only the conditional probability given the preceding word $P(w_n|w_{n-1})$. In other words, instead of computing the probability

$$
P(\text{blue}|\text{The water of Walden Ponf is so beautifully})
$$

we approximate it with the probability

$$
P(\text{blue}|\text{beautifully})
$$

When we use a bigram model to predict the conditional probability of the next word, we are thus making the following approximation:

$$
P(w_n|w_{1:n-1}) \approx P(w_n|w_{n-1})
$$

The assumption that the probability of a word depends only on the previous word is called a **Markov** assumption. Markov models are the class of probabilistic models that assume we can predict the probability of some future unit without looking too far into the past. We can generalize the bigram (which looks one word into the past) n-gram to the trigram (which looks two words into the past) and thus to the **n-gram** (which looks $n−1$ words into the past).

Let’s see a general equation for this n-gram approximation to the conditional probability of the next word in a sequence. We’ll use $N$ here to mean the n-gram size, so $N=2$ means bigrams and $N=3$ means trigrams. Then we approximate the probability of a word given its entire context as follows:

$$
P(w_n|w_{1:n-1}) \approx P(w_n|w_{n-N+1:n-1})
$$

Given the bigram assumption for the probability of an individual word, we can compute the probability of a complete word sequence:

$$
P(w_{1:n}) \approx \prod_{k=1}^n P(w_k|w_{k-1})
$$

### How to estimate probabilities

How do we estimate these bigram or n-gram probabilities? An intuitive way to estimate probabilities is called **maximum likelihood estimation** or **MLE**. We get the MLE estimate for the parameters of an n-gram model by getting counts from a corpus, and **normalizing** the counts so that they lie between 0 and 1. For probabilistic models, normalizing means dividing by some total count so that the resulting probabilities fall between 0 and 1 and sum to 1.

For example, to compute a particular bigram probability of a word $w_n$ given a previous word $w_{n-1}$, we’ll compute the count of the bigram $C(w_{n-1}w_n)$ and normalize by the sum of all the bigrams that share the same first word w_{n-1}:

$$
P(w_n|w_{n-1}) = \frac{C(w_{n-1}w_n)}{\sum_wC(w_{n-1}w)}
$$

We can simplify this equation, since the sum of all bigram counts that start with a given word $w_{n-1}$ must be equal to the unigram count for that word $w_{n-1}$ (the reader should take a moment to be convinced of this):

$$
P(w_n|w_{n-1}) = \frac{C(w_{n-1}w_n)}{\sum C(w_{n-1})}
$$

Let’s work through an example using a mini-corpus of three sentences. We’ll first need to augment each sentence with a special symbol $<s>$ at the beginning of the sentence, to give us the bigram context of the first word. We’ll also need a special end-symbol $</s>$.

<pre>
    &lt;s&gt; I am Sam &lt;/s&gt;
    &lt;s&gt; Sam I am &lt;/s&gt;
    &lt;s&gt; I do not like green eggs and ham &lt;/s&gt;
</pre>

Here are the calculations for some of the bigram probabilities from this corpus

$$
P(\text{I}|\text{<s>}) = \frac{2}{3} = 0.67
$$

$$
P(\text{Sam}|\text{<s>}) = \frac{1}{3} = 0.33
$$


$$
P(\text{</s>}|\text{Sam}) = \frac{1}{2} = 0.5
$$

$$
P(\text{do}|\text{I}) = \frac{1}{3} = 0.33
$$

For the general case of MLE n-gram parameter estimation:

$$
P(w_n|w_{n-N+1:n-1}) = \frac{C(w_{n-N+1:n-1} w_n)}{C(w_{n-N+1:n-1})}
$$

### Evaluating Language Models: Training and Test Sets

The best way to evaluate the performance of a language model is to embed it in an application and measure how much the application improves. Such end-to-end
evaluation is called **extrinsic evaluation**. Extrinsic evaluation is the only way to know if a particular improvement in the language model (or any component) is really going to help the task at hand. Thus for evaluating n-gram language models that are a component of some task like speech recognition or machine translation, we can compare the performance of two candidate language models by running the speech recognizer or machine translator twice, once with each language model, and seeing which gives the more accurate transcription.

Unfortunately, running big NLP systems end-to-end is often very expensive. Instead, it’s helpful to have a metric that can be used to quickly evaluate potential improvements in a language model. An **intrinsic evaluation** metric is one that measures the quality of a model independent of any application.

In order to evaluate any machine learning model, we need to have at least three distinct data sets: the **training set**, the **development set**, and the **test set**.

The **training set** is the data we use to learn the parameters of our model; for simple n-gram language models it’s the corpus from which we get the counts that we normalize into the probabilities of the n-gram language model. 

The **test set** is a different, held-out set of data, not overlapping with the training set, that we use to evaluate the model. We need a separate test set to give us an unbiased estimate of how well the model we trained can generalize when we apply it to some new unknown dataset. A machine learning model that perfectly captured the training data, but performed terribly on any other data, wouldn’t be much use when it comes time to apply it to any new data or problem! We thus measure the quality of an n-gram model by its performance on this unseen test set or test corpus.

How should we choose a training and test set? The test set should reflect the language we want to use the model for. If we’re going to use our language model for speech recognition of chemistry lectures, the test set should be text of chemistry lectures. If we’re going to use it as part of a system for translating hotel booking requests from Chinese to English, the test set should be text of hotel booking requests. If we want our language model to be general purpose, then the test set should be drawn from a wide variety of texts. In such cases we might collect a lot of texts from different sources, and then divide it up into a training set and a test set. It’s important to do the dividing carefully; if we’re building a general purpose model, we don’t want the test set to consist of only text from one document, or one author, since that wouldn’t be a good measure of general performance.

Thus if we are given a corpus of text and want to compare the performance of two different n-gram models, we divide the data into training and test sets, and train the parameters of both models on the training set. We can then compare how well the two trained models fit the test set.

But what does it mean to “fit the test set”? The standard answer is simple: whichever language model assigns a **higher probability** to the test set—which means it more accurately predicts the test set—is a better model. Given two probabilistic models, the better model is the one that better predicts the details of the test data, and hence will assign a higher probability to the test data. 

Since our evaluation metric is based on test set probability, it’s important not to let the test sentences into the training set. Suppose we are trying to compute the probability of a particular “test” sentence. If our test sentence is part of the training corpus, we will mistakenly assign it an artificially high probability when it occurs in the test set. We call this situation **training on the test set**. Training on the test set introduces a bias that makes the probabilities all look too high, and causes huge inaccuracies in **perplexity**, the probability-based metric we introduce below.

Even if we don’t train on the test set, if we test our language model on the test set many times after making different changes, we might implicitly tune to its characteristics, by noticing which changes seem to make the model better. For this reason, we only want to run our model on the test set once, or a very few number of times, once we are sure our model is ready.

For this reason we normally instead have a third dataset called a **development** test set or, **devset**. We do all our testing on this dataset until the very end, and then we test on the test set once to see how good our model is.

### Evaluating Language Models: Perplexity

We said above that we evaluate language models based on which one assigns a higher probability to the test set. A better model is better at predicting upcoming words, and so it will be less surprised by (i.e., assign a higher probability to) each word when it occurs in the test set. Indeed, a perfect language model would correctly guess each next word in a corpus, assigning it a probability of 1, and all the other words a probability of zero. So given a test corpus, a better language model will assign a higher probability to it than a worse language model. 

But in fact, we do not use raw probability as our metric for evaluating language models. The reason is that the probability of a test set (or any sequence) depends on the number of words or tokens in it; the probability of a test set gets smaller the longer the text. We’d prefer a metric that is per-word, normalized by length, so we could compare across texts of different lengths. The metric we use is, a function of probability called **perplexity**, is one of the most important metrics in NLP, used for evaluating large language models as well as n-gram models.

The perplexity (sometimes abbreviated as PP or PPL) of a language model on a test set is the inverse probability of the test set (one over the probability of the test set), normalized by the number of words (or tokens). For this reason it’s sometimes called the per-word or per-token perplexity. We normalize by the number of words $N$ by taking the Nth root. For a test set $W=w_1w_2 \cdots w_n$:

$$
\text{perplexity}(W) = P(w_1w_2\cdots w_N)^{-\frac{1}{N}} = \sqrt[N]{\frac{1}{P(w_1w_2\cdots w_N)}}
$$

Or we can use the chain rule to expand the probability of $W$:

$$
\text{perplexity}(W) = \sqrt[N]{\prod_{i=1}^N \frac{1}{P(w_i|w_1 \dots w_{i-1})}}
$$

Note that because of the inverse, the higher the probability of the word sequence, the lower the perplexity. Thus **the lower the perplexity of a model on the data, the better the model**.

The details of computing the perplexity of a test set $W$ depends on which language model we use. Here’s the perplexity of $W$ with a unigram language model
(just the geometric mean of the inverse of the unigram probabilities):

$$
\text{perplexity}(W) = \sqrt[N]{\prod_{i=1}^N \frac{1}{P(w_i)}}
$$

The perplexity of $W$ computed with a bigram language model is still a geometric mean, but now of the inverse of the bigram probabilities:

$$
\text{perplexity}(W) = \sqrt[N]{\prod_{i=1}^N \frac{1}{P(w_i|w_{i-1})}}
$$

### Sampling sentences from a language model

One important way to understand the knowledge embedded in a language model is to **sample from it**—that is, to generate text based on its learned probabilities.  

In probability and statistics, **sampling from a distribution** means selecting random points where more probable points are chosen more often. Applied to language models, this means generating sentences in a way that reflects the probability distribution the model has learned over sequences of words.  

For example, if a language model has been trained on a large corpus of English text, it might assign a high probability to the sentence:  

> "The sun rises in the east."  

while assigning a much lower probability to:  

> "Sun the rises east in."  

When we sample from the model, it is far more likely to generate the first sentence than the second one because the first aligns better with standard English grammar and word order.  

In practice, different sampling techniques can influence text generation. **Greedy decoding** always picks the most probable next word, leading to repetitive output. **Random sampling** injects more variety by picking words probabilistically. **Top-k sampling** restricts choices to the k most likely words, balancing coherence and diversity.  

By analyzing generated text, we gain insights into the model’s understanding of language—its strengths, biases, and limitations.

### Smoothing, Interpolation, and Backoff

There is a problem with using maximum likelihood estimates for probabilities: any finite training corpus will be missing some perfectly acceptable word sequences. That is, cases where a particular n-gram never occurs in the training data but appears in the test set. Perhaps our training corpus has the words _ruby_ and _slippers_ in it but just happens not to have the phrase _ruby slippers_.

These unseen sequences or **zeros** — sequences that don’t occur in the training set but do occur in the test set — are a problem for two reasons. First, their presence means we are underestimating the probability of word sequences that might occur, which hurts the performance of any application we want to run on this data. Second, if the probability of any word in the test set is 0, the probability of the whole test set is 0. Perplexity is defined based on the inverse probability of the test set. Thus if some words in context have zero probability, we can’t compute perplexity at all, since we can’t divide by 0.

The standard way to deal with putative “zero probability n-grams” that should really have some non-zero probability is called **smoothing** or **discounting**. Smoothing algorithms shave off a bit of probability mass from some more frequent events and give it to unseen events. Here we’ll introduce some simple smoothing algorithms: **Laplace (add-one) smoothing**, **stupid backoff**, and n-gram **interpolation**.

#### Laplace Smoothing

The simplest way to do smoothing is to add one to all the n-gram counts, before we normalize them into probabilities. All the counts that used to be zero will now have a count of 1, the counts of 1 will be 2, and so on. This algorithm is called Laplace smoothing. Laplace smoothing does not perform well enough to be used in modern n-gram models, but it usefully introduces many of the concepts that we see in other smoothing algorithms, gives a useful baseline, and is also a practical smoothing algorithm for other tasks like text classification.

Let’s start with the application of Laplace smoothing to unigram probabilities. Recall that the unsmoothed maximum likelihood estimate of the unigram probability of the word $w_i$ is its count $c_i$ normalized by the total number of word tokens $N$:

$$
P(w_i) = \frac{c_i}{N}
$$

Laplace smoothing merely adds one to each count (hence its alternate name **add-one smoothing**). Since there are $V$ words in the vocabulary and each one was incremented, we also need to adjust the denominator to take into account the extra $V$ observations. (What happens to our P values if we don’t increase the denominator?)

$$
P(w_i) = \frac{c_i + 1}{N + V}
$$

For add-one smoothed bigram counts, we need to augment the unigram count in the denominator by the number of total word types in the vocabulary $V$. We can see why this is in the following equation, which makes it explicit that the unigram count in the denominator is really the sum over all the bigrams that start with $w_{n−1}$. Since we add one to each of these, and there are $V$ of them, we add a total of $V$ to the denominator:

$$
P_{\text{Laplace}}(w_n|w_{n-1}) = \frac{C(w_{n-1}w_n) + 1}{\sum_w (C(w_{n-1}w_n) + 1)} = \frac{C(w_{n-1}w_n) + 1}{C(w_{n-1}) + V}
$$

#### Add-k smoothing

One alternative to add-one smoothing is to move a bit less of the probability mass from the seen to the unseen events. Instead of adding 1 to each count, we add a fractional count $k$ ($0.5$? $0.01$?). This algorithm is therefore called **add-k smoothing**.

$$
P_{\text{Add-k}}(w_n|w_{n-1}) = \frac{C(w_{n-1}w_n) + k}{C(w_{n-1}) + kV}
$$

Add-k smoothing requires that we have a method for choosing $k$; this can be done, for example, by optimizing on a **devset**. Although add-k is useful for some tasks (including text classification), it turns out that it still doesn’t work well for language modeling, generating counts with poor variances and often inappropriate discounts.

#### Language Model Interpolation

There is an alternative source of knowledge we can draw on to solve the problem of zero frequency n-grams. If we are trying to compute $P(w_n|w_{n−2}w_{n−1})$ but we have no examples of a particular trigram $w_{n−2}w_{n−1}w_n$, we can instead estimate its probability by using the bigram probability $P(w_n|w_{n-1})$. Similarly, if we don’t have counts to compute $P(w_n|w_{n-1})$, we can look to the unigram $P(w_n)$. In other words, sometimes using **less context** can help us generalize more for contexts that the model hasn’t learned much about.

The most common way to use this n-gram hierarchy is called **interpolation**: computing a new probability by interpolating (weighting and combining) the trigram, bigram, and unigram probabilities. In simple linear interpolation, we combine different order n-grams by linearly interpolating them. Thus, we estimate the trigram probability $P(w_n|w_{n−2}w_{n−1})$ by mixing together the unigram, bigram, and trigram probabilities, each weighted by a $\lambda$:

$$
\hat{P}(w_n|w_{n-2}w_{n-1}) = \lambda_1 P(w_n) 
                             + \lambda_2 P(w_n|w_{n-1}) 
                             + \lambda_3 P(w_n|w_{n-1}w_{n-2})

$$

#### Stupid Backoff

An alternative to interpolation is **backoff**. In a backoff model, if the n-gram we need has zero counts, we approximate it by backing off to the (n-1)-gram. We continue backing off until we reach a history that has some counts. For a backoff model to give a correct probability distribution, we have to **discount** the higher-order n-grams to save some probability mass for the lower order n-grams. In practice, instead of discounting, it’s common to use a much simpler non-discounted backoff algorithm called **stupid backoff**.

